# Preprocessing

In [13]:

import sklearn
assert sklearn.__version__ >= "0.20"
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline 
import matplotlib as mpl
import matplotlib.pyplot as plt

In [14]:
input_df = pd.read_csv("input_df.csv")
input_df_pca = pd.read_csv("input_df_pca.csv")
output_df_class = pd.read_csv("output_df_class.csv")
output_df_class.drop(columns=["Unnamed: 0"], inplace=True)
input_df_pca.drop(columns=["Unnamed: 0"], inplace=True)
input_df.drop(columns=["Unnamed: 0"], inplace=True)

In [15]:
input_dict = {"Not PCA": input_df.copy(), "PCA": input_df_pca.copy()}
Y = output_df_class.copy()

target_names = ["bad_class", "good_class", "very_good_class"]
class_counter = 0
Y["multiclass"] = np.where(pd.isna(Y["class_bad"]), np.nan, 0)
for c in ["class_bad", "class_good", "class_very_good"]:
    Y[c] = np.where(Y[c] == 1, class_counter, 0)
    class_counter += 1
Y["multiclass"] = Y.sum(axis=1) + Y["multiclass"]
Y["multiclass"] = np.where(pd.isna(Y["multiclass"]), -1, Y["multiclass"])
Y = np.array(Y["multiclass"])

df_results = pd.DataFrame(columns=["Method", "Input" ])

In [16]:
def delete_unknown(y_test, y_pred):
    """Takes y_true and y_pred and supress in both the values corresponding to a -1 for y_test"""

    y_test.resize((y_test.shape[0], 1))
    y_pred.resize((y_pred.shape[0], 1))
    array = np.concatenate((y_test, y_pred), axis=1)
    array = array[np.logical_not(array[:, 0] == -1)]
    y_t = array[:, 0].astype("float64")
    y_p = array[:, 1].astype("float64")
    return y_t, y_p

# Self training

In [17]:
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    r2_score,
    f1_score,
    recall_score,
    precision_score,
)

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

In [18]:
n_splits = 10

scores_f1_micro = np.empty(n_splits)
scores_f1_macro = np.empty(n_splits)
scores_precision_macro = np.empty(n_splits)
scores_recall_macro = np.empty(n_splits)

scores_accuracy = np.empty(n_splits)

scores_r2 = np.empty(n_splits)

# class_weight_dict = {0: 0.4,1:0.2,2:0.4}
# base_classifier = SVC(C=10.0, probability=True, gamma='scale', random_state=42, class_weight=class_weight_dict, tol = 1e-6)

models = {
    "Bagging Classifier": BaggingClassifier(
        random_state=42, max_features=0.8, max_samples=0.8, n_estimators=150
    ),
    "K-Nearest Neighbors Classifier (distance)": KNeighborsClassifier(
        n_neighbors=4, p=1, weights="distance"
    ),
    "K-Nearest Neighbors Classifier (uniform)": KNeighborsClassifier(
        n_neighbors=3, weights="uniform"
    ),
    "Support Vector Classifier (rbf)": SVC(
        C=100.0, probability=True, gamma=0.01, kernel="rbf"
    ),
    "Support Vector Classifier (poly)": SVC(
        C=10.0, probability=True, degree=3, gamma=0.5, kernel="poly"
    ),
    "Support Vector Classifier (linear)": SVC(
        C=10.0, probability=True, kernel="linear"
    ),
    "Decision Tree Classifier": DecisionTreeClassifier(
        max_depth=15, min_samples_split=2
    ),
    "Random Forest Classifier": RandomForestClassifier(
        random_state=42, max_depth=10, min_samples_split=5, n_estimators=10
    ),
}


for model_name, base_classifier in models.items():
    self_training_model = SelfTrainingClassifier(
        base_classifier, threshold=0.7,
    )
    skfolds = StratifiedKFold(n_splits=n_splits)


    for key, X in input_dict.items():

        conf_mat = np.zeros((3, 3))
        for fold, (train_index, test_index) in enumerate(skfolds.split(X, Y)):
            X_train = X.iloc[train_index, :]
            y_train = Y[train_index]
            X_test = X.iloc[test_index, :]
            y_test = Y[test_index]

            self_training_model.fit(X_train, y_train)
            y_pred = self_training_model.predict(X_test)

            y_test, y_pred = delete_unknown(y_test, y_pred)

            scores_accuracy[fold] = accuracy_score(y_test, y_pred)

            scores_r2[fold] = r2_score(y_test, y_pred)

            scores_accuracy[fold] = accuracy_score(y_test, y_pred)
            scores_f1_micro[fold] = f1_score(y_test, y_pred, average="micro")
            scores_f1_macro[fold] = f1_score(y_test, y_pred, average="macro")
            scores_precision_macro[fold] = precision_score(
                y_test,
                y_pred,
                average="macro",
                zero_division=np.nan,
            )
            scores_recall_macro[fold] = recall_score(y_test, y_pred, average="macro")

            matrix = confusion_matrix(list(y_test), list(y_pred))
            conf_mat += matrix


        df_results = df_results._append(
            {
                "Method": model_name,

                "Input": key,

                "R2": "{} +- {}".format(
                    round(scores_r2.mean(), 3), round(scores_r2.std(), 3)

                ),

                "Accuracy": "{} +- {}".format(
                    round(scores_accuracy.mean(), 3), round(scores_accuracy.std(), 3)

                ),
                "F1-score (micro)": "{} +- {}".format(
                    round(scores_f1_micro.mean(), 3), round(scores_f1_micro.std(), 3)

                ),
                "F1-score (macro)": "{} +- {}".format(
                    round(scores_f1_macro.mean(), 3), round(scores_f1_macro.std(), 3)

                ),
                "Precision (macro)": "{} +- {}".format(
                    round(scores_precision_macro.mean(), 3), round(scores_precision_macro.std(), 3)

                ),
                "Recall (macro)": "{} +- {}".format(
                    round(scores_recall_macro.mean(), 3), round(scores_recall_macro.std(), 3)

                ),
                
                "Confusion Matrix": conf_mat,
            },
            ignore_index=True,
        )


df_results.sort_values("R2", ignore_index=True)

Bagging Classifier 0
Bagging Classifier 1
Bagging Classifier 2
Bagging Classifier 3


KeyboardInterrupt: 